In [ ]:
#SUMMARY

#web: server-client: url
#동적 페이지: URL 변화 없이 페이지의 데이터 수정: json(str) > response.json() > DataFrame
#정적 페이지: URL 변화 있이 페이지의 데이터 수정: html(str) > BeautifulSoup > css-selector > DataFrame
#selenium: 웹브라우져를 python 코드로 컨트롤해서 데이터 수집
#requests(동적페이지,API)>requests(정적페이지) > selenium

#웹크롤링 절차
#1. 웹서비스분석(개발자도구): URL
#2. request(URL) > response(data): data(json(str),html(str))
#3. data(json(str), html(str)) > 파씽: response.json(), BeautifulSoup(css-selector) > DataFrame

#request 할때 401, 403, 500 등등의 에러가 발생하는 경우 > headers 수정해서 데이터 요청(user-agent, refere)
#API 이용: request token 수집 후 크롤링

### Zigbang 원룸 매물 데이터 수집
- 동 이름(입력) > 매물 데이터(출력)

In [1]:
import pandas as pd
import requests

#### Process
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

#### 1. 동이름으로 위도 경도 구하기

URL Decoder / Encoder : https://meyerweb.com/eric/tools/dencoder/

In [4]:
#1. URL
address = '개포동' 
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸'

#2. request > response : json(str)
response = requests.get(url)

#3. json(str) > lat, lng
data = response.json()['items'][0] 
lat, lng = data['lat'], data['lng'] 

In [17]:
response.text

'{"success":true,"code":"200","items":[{"id":3735,"type":"address","name":"개포동","hint":"","description":"서울시 강남구 개포동","lat":37.48192596435547,"lng":127.05744171142578,"zoom":5,"polygon":[],"_score":null,"_source":{"name_length":3,"local1":"서울시","local2":"강남구","local3":"개포동","web_level":15,"web_lat":37.48192596435547,"web_lng":127.05744171142578,"app_level":15,"app_lat":37.48192596435547,"app_lng":127.05744171142578,"법정동코드":"1168010300"},"zoom_level":{"google":15,"daum":4},"zoom_level_v2":{"app":5,"web":4}},{"id":129,"type":"subway","name":"개포동역","hint":"수인분당선","description":"서울특별시 강남구 개포동","lat":37.4891159,"lng":127.065987,"zoom":3,"polygon":[],"_score":null,"_source":{"name_length":4,"local1":"서울특별시","local2":"강남구","local3":"개포동","suggestions":[],"suggestions_insensitive":[],"distance":1000},"zoom_level":{"google":14,"daum":4},"zoom_level_v2":{"app":6,"web":4}}],"next":null,"limit":0}'

#### 2. 위도 경도로 geohash 알아내기
- geohash2: pip install geohash2

In [5]:
#  !pip install geohash2

In [6]:
import geohash2

In [7]:
#precision: 클수록 영역이 작아짐(숫자가 작을수록 큰 영역을 의미함)
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydm5'

#### 3. geohash로 매물 아이디 가져오기

In [18]:
# 한글 들어간건 디코드 해서 카피 or 우클릭 copy value
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
response = requests.get(url)
response

<Response [200]>

In [19]:
#items id만 필요한 정보임을 알 수 있다.
response.text

'{"clusters":[],"items":[{"lat":37.44458690681313,"lng":127.05876109902758,"item_id":35600180},{"lat":37.46292198091118,"lng":127.04997457514203,"item_id":35695563},{"lat":37.4691833864648,"lng":127.04620331368294,"item_id":35611559},{"lat":37.469155381883034,"lng":127.04623920493496,"item_id":35679182},{"lat":37.47016650899742,"lng":127.04568609315243,"item_id":35335715},{"lat":37.47030495684337,"lng":127.04659655066689,"item_id":35608899},{"lat":37.46988878789614,"lng":127.04690145985205,"item_id":35529990},{"lat":37.46992123799065,"lng":127.0464396451733,"item_id":35632836},{"lat":37.4699697660604,"lng":127.04789029554793,"item_id":35349519},{"lat":37.47048068469541,"lng":127.04786159737343,"item_id":35663883},{"lat":37.47009586939562,"lng":127.04742862552517,"item_id":35526746},{"lat":37.47089221498099,"lng":127.04886053930146,"item_id":35703075},{"lat":37.470612872814684,"lng":127.04795519482731,"item_id":35699198},{"lat":37.47069508871911,"lng":127.047011670976,"item_id":35366884

In [20]:
data = response.json()['items']
ids = [item['item_id'] for item in data]
ids[:5]

[35600180, 35695563, 35611559, 35679182, 35335715]

#### 4. 매물 아이디로 매물 정보 가져오기

In [21]:
url = f'https://apis.zigbang.com/v2/items/list' 
params = {                                  # item 머시기 payload를 통해 확인
          'domain': 'zigbang', 
          'withCoalition': 'true', 
          'item_ids': ids[:900], # 직방은 아이템 데이터의 갯수를 999개까지 사용 가능 
          } 

response = requests.post(url, params) #post 방식, 영문 숫자 데이터만 있어서 dumps 사용 x
response

<Response [200]>

In [25]:
#items 데이터 딕셔너리 -> 로우 데이터 하나: 데이터 프레임 만들기
data = response.json()['items']
df = pd.DataFrame(data)
df.tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new,contract
97,None,35688621,https://ic.zigbang.com/ic/items/35688621/1.jpg,월세,월세,3000,30,16.53,"{'m2': 16.53, 'p': '5'}","{'m2': 16.53, 'p': '5'}",...,True,원룸,[추천],서울시 강남구 개포동,None,None,10,2023-03-03T17:19:16+09:00,False,
98,None,35672417,https://ic.zigbang.com/ic/items/35672417/1.jpg,전세,전세,26500,0,50.58,"{'m2': 50.58, 'p': '15.3'}","{'m2': 43.26, 'p': '13.1'}",...,True,빌라,[],서울시 강남구 개포동,None,None,20,2023-03-02T18:17:46+09:00,False,


In [29]:
#필요한 컬럼만 필터링
#1. ... 먼저 보여주기
pd.options.display.max_columns = 50
df.tail(1)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new,contract
98,None,35672417,https://ic.zigbang.com/ic/items/35672417/1.jpg,전세,전세,26500,0,50.58,"{'m2': 50.58, 'p': '15.3'}","{'m2': 43.26, 'p': '13.1'}",None,None,4,4,5,반전세가능💖예쁨주의💖쓰리룸💖대출가능,None,05,강남구 개포동,"{'lat': 37.479868459770614, 'lng': 127.0482737...",False,True,빌라,[],서울시 강남구 개포동,None,None,20,2023-03-02T18:17:46+09:00,False,


In [30]:
df.columns

Index(['section_type', 'item_id', 'images_thumbnail', 'sales_type',
       'sales_title', 'deposit', 'rent', 'size_m2', '공급면적', '전용면적', '계약면적',
       'room_type_title', 'floor', 'floor_string', 'building_floor', 'title',
       'is_first_movein', 'room_type', 'address', 'random_location', 'is_zzim',
       'status', 'service_type', 'tags', 'address1', 'address2', 'address3',
       'manage_cost', 'reg_date', 'is_new', 'contract'],
      dtype='object')

In [ ]:
# 데이터 전처리


In [36]:
#원하는 컬럼만 출력
columns = ['item_id',  'sales_type', 'deposit','rent', 'size_m2', 
           'floor','building_floor', 'title','address','status', 
           'service_type', 'tags','address1','manage_cost']
df = df[columns]
# 데이터 전처리(검색지역 외 지역 필터링)
df = df[df['address'].str.contains(address)].reset_index(drop=True)
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,title,address,status,service_type,tags,address1,manage_cost
63,35688621,월세,3000,30,16.53,2,4,☎☎보증금대출 불가! 관리비에 모든 공과금 포함! 풀옵션,강남구 개포동,True,원룸,[추천],서울시 강남구 개포동,10
64,35672417,전세,26500,0,50.58,4,5,반전세가능💖예쁨주의💖쓰리룸💖대출가능,강남구 개포동,True,빌라,[],서울시 강남구 개포동,20


In [ ]:
# 꿀팁: 학습방법 회사에서 일을 하기 위한 수준으로
'''
예를 들어 pandas 공부를 해야한다면 pandas 공식 사이트에 간다.-> doucumentation 해보기, 10 minutes to pandas
mysql도 마찬가지
파이썬 공부는 pep documents
문법(잘못작성하면 에러, 코드 실행x), 컨벤션(에러밸생x, 코드실행o)
pep 20(파이썬철학), pep8(코딩스타일 가이드)
flake8: 코드의효율성 체크
'''

In [37]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
